# Objektově orientované programování

**NB:** Obsah tohoto notebooku navazuje na výklad o morfologickém značkování v notebooku `pos_tagging.ipynb`.

In [ ]:
# příprava dat, se kterými budeme pracovat

def vert_sents(path):
    sents = []
    with open(path) as file:
        for line in file:
            line = line.strip("\r\n")
            if line == "<s>":
                sent = []
            elif line == "</s>":
                sents.append(sent)
            else:
                word, _, tag = line.split("\t")
                sent.append((word, tag[0]))
    return sents

train = vert_sents("/home/lukes/edu/python/pdt3.train.vrt")
sent = "Běží liška k táboru .".split()

## K čemu jsou vůbec objekty dobré?

Princip: někdy jsou data intimně spjata s funkcemi, které s nimi pracují. Např. tagovací funkce vždy bude potřebovat natrénovaný model, na jehož základě tagování provede. Pak je nešikovné mít data reprezentující tento model (tj. typicky podmíněnou frekvenční distribuci) uložená stranou a muset je pokaždé tagovací funkci předávat:

In [ ]:
import nltk

def train_uni(data):
    """Natrénuje unigramový tagger."""
    cfd = nltk.ConditionalFreqDist()
    for sent in data:
        for word, tag in sent:
            cfd[word.lower()][tag] += 1
    return cfd

def tag_uni(sent, uni_cfd):
    """Otaguje větu pomocí unigramového taggeru."""
    tagged_sent = []
    for word in sent:
        try:
            tag = uni_cfd[word.lower()].max()
        except ValueError:
            # pokud tvar neznáme, přiřadíme mu tag "N"
            tag = "N"
        tagged_sent.append((word, tag))
    return tagged_sent

In [ ]:
model = train_uni(train)
tag_uni(sent, model)

Zde musíme `tagger` pokaždé explicitně funkci `tag_uni()` předávat jako argument, což zavání průšvihem. Může se nám třeba stát, že si omylem do proměnné `tagger` uložíme něco jiného, nebo že ho zapomeneme funkci předat, nebo dokonce omylem zavoláme unigramovou tagovací funkci s bigramovým taggerem, což povede k výsledkům, které budou špatně, ale nemusíme si toho nutně všimnout. Bylo by tedy dobré vytvořit objekt, na který "navěsíme" jak funkci, tak tagger, aby bylo jasné, že patří k sobě a že se mají používat společně.

Nejprve je potřeba definovat nový typ objektu, tzv. **třídu** (`class`), u níž stanovíme, jaké má přidružené funkce (**metody**) a jakým způsobem se vytvářejí nové instance této třídy (tj. to, čemu běžně říkáme objekty). Ze stávajících funkcí `train_uni()` a `tag_uni()` uděláme metody jednoduše tak, že je nakopírujeme pod definici třídy (tj. odsazené) a přidáme jim na začátek seznamu argumentů jeden navíc, který se tradičně pojmenovává **`self`**. Tento argument je odkaz na instanci objektu, na němž metodu voláme. Z hlediska syntaktického zápisu je to to, co je "před tečkou": voláme-li metodu `bar()` na objektu `foo`, zapíšeme to `foo.bar()` a uvnitř metody `bar()` budeme mít přístup k objektu `foo` přes lokální proměnnou `self`.

In [ ]:
class UnigramTagger:
    def train_uni(self, data):
        """Natrénuje unigramový tagger."""
        cfd = nltk.ConditionalFreqDist()
        for sent in data:
            for word, tag in sent:
                cfd[word.lower()][tag] += 1
        return cfd

    def tag_uni(self, sent, uni_cfd):
        """Otaguje větu pomocí unigramového taggeru."""
        tagged_sent = []
        for word in sent:
            try:
                tag = uni_cfd[word.lower()].max()
            except ValueError:
                tag = "N"
            tagged_sent.append((word, tag))
        return tagged_sent

Bohužel s takto minimálními úpravami se tagger používá stále stejně nešikovně: musíme si tagger natrénovaný pomocí funkce `train_uni()` uložit do proměnné a pak ho explicitně předat funkci `tag_uni()`. Jediné, čeho jsme dosáhli, je to, že jsme nějakým způsobem naznačili, že obě funkce patří k sobě (protože jsou definované jako metody na jedné třídě).

In [ ]:
tagger_obj = UnigramTagger()
model = tagger_obj.train_uni(train)
tagger_obj.tag_uni(sent, model)

Jinak je použití identické tomu, jak se používají funkce, jen zápis je delší, protože teď se musíme trápit navíc s vytvářením objektu a voláním metod na něm (místo samostatných funkcí). Co s tím? Místo toho, abychom v metodě `train_uni()` natrénovaný tagger vraceli (pomocí **`return`**), **"odložíme" si ho přes `self` na objekt**, odkud k němu pak budou mít přístup i další metody. Taky můžeme odstranit sufixy `_uni` na metodách (když jsou metody definované na třídě, která se jmenuje `UnigramTagger`, je asi celkem zbytečné "unigramovost" tohoto taggeru naznačovat znovu ve jménu každé metody).

In [ ]:
class UnigramTagger:
    def train(self, data):
        """Natrénuje unigramový tagger."""
        # vytvoříme podmíněnou frekvenční distribuci
        cfd = nltk.ConditionalFreqDist()
        for sent in data:
            for word, tag in sent:
                cfd[word.lower()][tag] += 1
        # POZOR, tentokrát ne return; místo toho si podm. frek. dist.
        # "odložíme" na aktuální objekt (přes self)
        self._model = cfd
        # podtržítko na začátku atributu značí, že daný atribut v rámci
        # objektu zamýšlíme jako "soukromý". tím naznačujeme uživateli,
        # že na něj nemá sám ručně sahat, že je to implementační detail,
        # za nějž jsme zodpovědní a s nímž za normálních okolností
        # pracujeme pouze my jako původní autoři definice třídy

    def tag(self, sent):
        """Otaguje větu pomocí unigramového taggeru."""
        tagged_sent = []
        for word in sent:
            try:
                # zde už nepředáváme tagger jako argument, ale vezmeme
                # si ho z atributu self._model
                tag = self._model[word.lower()].max()
            except ValueError:
                tag = "N"
            tagged_sent.append((word, tag))
        return tagged_sent

Použití takto upraveného objektu je hned elegantnější:

In [ ]:
tagger_obj = UnigramTagger()
tagger_obj.train(train)
tagger_obj.tag(sent)

Poslední kosmetická úprava: v aktuální verzi se nám může stát, že někdo vytvoří tagger a pak se pokusí otagovat větu dřív, než jej natrénuje:

In [ ]:
tagger_obj = UnigramTagger()
tagger_obj.tag(sent)

Vzhledem k tomu, že tagger bez natrénovaného tagovacího modelu nedává moc smysl, mohli bychom svou třídu ještě doladit tak, že donutíme uživatele, aby si tagger natrénoval při inicializaci nového objektu. Když vytváříme novou instanci objektu pomocí toho, že zavoláme jméno třídy jako funkci (připojíme za něj závorky, tj. `UnigramTagger()`), pod kapotou se ve skutečnosti volá speciální metoda `__init__()` na třídě `UnigramTagger`. Ta má nějakou defaultní podobu, která nedělá nic zajímavého a která se použije ve chvíli, kdy nespecifikujeme žádnou vlastní implementaci.

Nám by se ale hodilo, aby se v rámci `__init__()` provedlo trénování taggeru. Stačí tedy vlastně přejmenovat metodu `train()` na `__init__()`...

In [ ]:
class UnigramTagger:
    def __init__(self, data):
        """Inicializuje unigramový tagger, včetně natrénování modelu."""
        cfd = nltk.ConditionalFreqDist()
        for sent in data:
            for word, tag in sent:
                cfd[word.lower()][tag] += 1
        self._model = cfd

    def tag(self, sent):
        """Otaguje větu pomocí unigramového taggeru."""
        tagged_sent = []
        for word in sent:
            try:
                tag = self._model[word.lower()].max()
            except ValueError:
                tag = "N"
            tagged_sent.append((word, tag))
        return tagged_sent

... a konstruktoru pak předat příslušná trénovací data:

In [ ]:
tagger_obj = UnigramTagger(train)
tagger_obj.tag(sent)

Teď už můžeme být spokojeni :)

### Třídy bez metod

Někdy se hodí mít objekt na modelování nějakého strukturovaného typu dat, např. token v korpusu, který se skládá z wordu, lemmatu a tagu. Můžeme tedy napsat třídu, která tyto údaje "obalí" do jednoho objektu, jehož atributy jsou jednoduše přístupné přes tečku:

In [ ]:
class Token:
    
    def __init__(self, word, lemma, tag):
        self.word = word
        self.lemma = lemma
        self.tag = tag
        
t = Token("koček", "kočka", "N")
t.tag

To je ale poměrně dost kódu na to, že chceme jen navěsit různé spřízněné údaje na jeden objekt. Navíc takový objekt nemá defaultně žádnou přívětivou reprezentaci, která by nám umožnila si jednoduše prohlédnout jeho obsah při interaktivní práci:

In [ ]:
t

To bychom museli na třídě navíc definovat ještě speciální metodu `__repr__()`, která stanovuje, jak vypadá tištěná reprezentace objektu:

In [ ]:
class Token:
    
    def __init__(self, word, lemma, tag):
        self.word = word
        self.lemma = lemma
        self.tag = tag
        
    def __repr__(self):
        return (
            # literál řetězce lze pro lepší čitelnost
            # rozdělit na několik dílčích, Python si
            # několik kratších po sobě jdoucích 
            # řetězcových literálů při načítání pospojuje
            # dohromady, takže např. z "a" "b" "c" se
            # stane "abc"
            f"{self.__class__.__name__}(
            f"word={self.word!r}, "
            f"lemma={self.lemma!r}, "
            f"tag={self.tag!r})"
        )
        
Token("koček", "kočka", "N")

Což je v součtu čím dál tím víc kódu. Pro takovéto účely je tedy lepší použít [modul `dataclasses`](https://docs.python.org/3/library/dataclasses.html), s jehož pomocí takovouto "obalovací" třídu vytvoříme raz dva:

In [ ]:
from dataclasses import dataclass

@dataclass
class Token:
    word: str
    lemma: str
    tag: str

S některými syntaktickými prvky v předchozí buňce jsme se ještě nesetkali. Syntax `@dataclass` je tzv. [dekorátor](https://realpython.com/primer-on-python-decorators/) a je ekvivalentní následujícímu kódu:

In [ ]:
class Token:
    word: str
    lemma: str
    tag: str

Token = dataclass(Token)

Funkce `dataclass()` za nás právě doplní implementaci metod `.__init__()`, `.__repr__()` a dalších potřebných. Dekorátor je jen elegantnější, specializovaná syntax, jak tuto funkci na naši třídu `Token` zavolat a výsledek rovnou přeuložit pod stejným jménem (`Token`).

Co se syntaxe `word: str` týče, je to způsob, jak v Pythonu (nepovinně) anotovat typy proměnných a atributů. Tuto informaci umí využívat některé externí nástroje, které vám můžou pomoct např. ohlídat, abyste funkci, která jako argument očekává řetězec, nezavolali omylem místo řetězce třeba s číslem (tzv. [*type checking*](https://realpython.com/python-type-checking/)).

Python samotný je ale ignoruje, bez těch externích nástrojů slouží tyto anotace jen jako nápověda pro programátory, případně je mohou využívat některé knihovny pro různé účely. Informaci o anotacích lze totiž získat pomocí speciálního atributu `.__annotations__`:

In [ ]:
Token.__annotations__

Funkce `dataclass` ji např. využívá tak, že typovou anotaci bere jako signál, že má daný atribut zahrnout do metody `.__init__()`, kterou pro nás vygeneruje. Z toho všeho plyne, že tuto novou třídu `Token` můžeme používat úplně stejně jako tu, co jsme si napsali ručně:

In [ ]:
t = Token("koček", "kočka", "N")
t.tag

In [ ]:
t

Inicializaci i hezky čitelnou reprezentaci celého objektu máme zadarmo, bez práce.

Pomocí funkcí `astuple()` a `asdict()` je pak snadné takovou třídu převést na odpovídající n-tici nebo slovník, kdykoli je to potřeba:

In [ ]:
from dataclasses import astuple, asdict

In [ ]:
astuple(t)

In [ ]:
asdict(t)

Třídy vytvořené pomocí dekorátoru `@dataclass` jsou defaultně modifikovatelné, tj. hodnoty atributů lze po vytvoření instance libovolně měnit:

In [ ]:
t.tag = "chachacha"
t

Někdy je ale vhodnější či prostě bezpečnější dodatečné modifikace zakázat. Toho lze docílit pomocí tzv. [zmrazených instancí](https://docs.python.org/3/library/dataclasses.html#frozen-instances): stačí dekorátor zavolat s parametrem `frozen=True`, tj. `@dataclass(frozen=True)`.

Jiná možnost je použít tzv. [pojmenované n-tice](https://docs.python.org/3/library/typing.html#typing.NamedTuple), angl. *named tuples*. Ty jsou automaticky nemodifikovatelné a zachovávají si i další vlastnosti n-tic bez nutnosti konverze pomocí funkce `astuple()`. Pokud upravujete existující kód, kde jste doposud používali n-tice, ale chcete jejich prvky pojmenovat, pojmenované n-tice jsou z hlediska zpětné kompatibility velmi dobrá volba.

Nové třídy pojmenovaných n-tice se nevytvářejí pomocí dekorátorů, ale pomocí dědění (viz následující oddíl):

In [ ]:
from typing import NamedTuple

class Token(NamedTuple):
    word: str
    lemma: str
    tag: str

In [ ]:
t = Token("koček", "kočka", "N")
t

Stejně jako normální n-tice jsou pojmenované n-tice nemodifikovatelné, takže pokus nastavit u existující instance jiný tag selže:

In [ ]:
t.tag = "chachacha"

A stejně jako normální n-tice lze s pojmenovanými n-ticemi pracovat jako s kolekcemi, tj. destrukturovat je, snadno je převádět na jiné kolekce atp.

In [ ]:
word, lemma, tag = t
word

In [ ]:
list(t)

In [ ]:
set(t)

## Spřízněné typy objektů (třídy): dědění (angl. *inheritance*)

Třídu `BigramTagger` můžeme vytvořit analogicky s použitím funkcí, které jsme vymysleli na předchozích hodinách (zatím se budeme soustředit jen na inicializaci taggeru, tj. trénování):

In [ ]:
class BigramTagger:
    def __init__(self, data):
        """Inicializuje bigramový tagger, včetně natrénování modelu."""
        cfd = nltk.ConditionalFreqDist()
        for sent in data:
            for pos, (word, tag) in enumerate(sent):
                # na začátku věty je předchozí tag None, ...
                if pos == 0:
                    prev_tag = None
                # ... kdekoli jinde je to prostě... předchozí tag :)
                else:
                    prev_tag = sent[pos-1][1]
                # stanovíme "kontext", v němž chceme zaznamenat, že jsme
                # v rámci trénovacích dat právě pozorovali daný tag
                context = (prev_tag, word.lower())
                cfd[context][tag] += 1
        self._model = cfd

Jen připomenu, že výstupem funkce `enumerate()` je generátor, jehož výsledek po rozbalení vypadá strukturně nějak takto:

In [ ]:
list(enumerate(train[0]))

Když srovnáme metody `__init__()` v `UnigramTagger`u a `BigramTagger`u, zjistíme, že obsahují mnoho stejného či podobného kódu:

- vytvoření prázdné podmíněné frekvenční distribuce
- pak procházíme trénovací data větu po větě
- každou větu procházíme pozici po pozici
- pro každou pozici stanovíme nějaký kontext, který vstupuje do podmíněné frekvenční distribuce jako podmínka, za níž zaznamenáme, že jsme pozorovali přítomnost nějakého tagu
- nakonec podmíněnou frekvenční distribuci uložíme na objekt do atributu `_model`

Je tu v podstatě jen jeden rozdíl: liší se definice unigramového a bigramového kontextu (aktuální slovní tvar vs. předchozí tag + aktuální slovní tvar).

Z hlediska údržby kódu a dalšího případného přidávání funkcionality je nešikovné mít velmi podobný kód nakopírovaný ve dvou třídách -- jakékoli úpravy bychom museli dělat v obou zároveň, na což se lehko zapomíná. Výhodnější by tedy bylo definovat si nějakou nadřazenou třídu `NgramTagger`, která implementuje tuto trénovací logiku obecně, a za které pak můžou třídy `UnigramTagger` a `BigramTagger` dědit takovým způsobem, aby stačilo implementovat jen metody, které jsou pro ně specifické (tj. v našem případě nejspíš jen metodu, která stanoví relevantní kontext, který použijeme jako podmínku do podmíněné frekvenční distribuce).

Celé to může vypadat např. takto:

In [ ]:
class NgramTagger:
    """Základní abstraktní třída pro n-gramové taggery.
    
    Mluvíme o ní jako o abstraktní proto, že nemá smysl vytvářet
    instance této třídy, klíčová metoda ``get_context()`` na ní totiž
    není implementovaná, protože její implementace závisí na tom, zda
    chceme tagger unigramový, bigramový, atd.
    
    Tato třída tedy slouží výhradně k tomu, abychom na jejím základě
    vytvořili podřazené třídy, které z ní dědí a které tyto metody
    již konkrétně implementují.
    
    """
    def __init__(self, data):
        """Natrénuje tagger."""
        cfd = nltk.ConditionalFreqDist()
        for sent in data:
            # (i, (word, tag)) ← (0, ('Jaderné', 'A'))
            for i, (word, tag) in enumerate(sent):
                if tag is None:
                    continue
                ctx = self.get_context(i, word, sent)
                cfd[ctx][tag] += 1
        self._model = cfd

    def get_context(self, i, word, sent):
        # tímto naznačujeme, že třída je abstraktní: slouží jen k tomu,
        # aby z ní dědily další třídy, které již musí tyto metody konkrétně
        # implementovat, aby šly použít (= aby z nich šly vytvářet instance,
        # tj. samotné objekty)
        raise NotImplementedError
        # pozn.: vyvolání chyby NotImplementedError je jen jeden ze způsobů
        # jak upozornit na to, že tato třída neslouží k přímému použití
        # (ten nejjednodušší); sofistikovanější možnosti nabízí modul abc,
        # viz např. https://pymotw.com/3/abc/
        
# tento zápis znamená, že UnigramTagger dědí ze třídy NgramTagger.
# v praxi to znamená, že přebírá všechny vlastnosti této nadřazené
# třídy, pokud je explicitně nenahradíme (což můžeme udělat tak, že
# v rámci podřazené třídy definujeme vlastnost -- třeba metodu --
# se stejným jménem).
class UnigramTagger(NgramTagger):
 
    def get_context(self, i, word, sent):
        # u unigramového taggeru větu ke stanovení kontextu
        # nepotřebujeme, stačí nám současné slovo
        return word.lower()
    
class BigramTagger(NgramTagger):
  
    def get_context(self, i, word, sent):
        # u bigramového taggeru z věty vytáhneme hodnotu předchozího tagu
        # (jsme-li na začátku věty, použijeme místo ní hodnotu None)
        prev_tag = None if i == 0 else sent[i-1][1]
        return prev_tag, word.lower()

Další výhodou tohoto přístupu je, že nyní máme i ve zdrojovém kódu reprezentovaný vztah mezi oběma typy taggerů: už to nejsou samostatné, nesouvisející třídy, ale jsou propojené přes základní třídu `NgramTagger`. Díky tomu si může jiný programátor dovodit, že spolu nějakým způsobem souvisí, že jsou spřízněné, a že by se tedy měly chovat podobně. K základní třídě (či třídám, pokud je hierarchie dědění hlubší, což být může) se dostaneme přes speciální atribut `__bases__`:

In [ ]:
UnigramTagger.__bases__

In [ ]:
BigramTagger.__bases__

In [ ]:
ut = UnigramTagger(train)
# pro ověření se podíváme třeba na frekvenční distribuci tagů
# pozorovaných v kontextu tvaru "stát"
ut._model["stát"]

In [ ]:
bt = BigramTagger(train)
# pro ověření se podíváme třeba na frekvenční distribuci tagů
# pozorovaných v kontextu tvaru "stát" a předchozího tagu "P"
# (zájmeno)
bt._model[("P", "stát")]

## Metoda `.tag()` a *backoff*

Nyní doplníme i metodu `.tag()` a možnost propojit taggery přes *backoff*. Možných postupů je pochopitelně víc, my se budeme snažit hlavně o názornost a srozumitelnost spíš než o efektivitu. Níže jsou změny oproti předchozí verzi kódu označeny pomocí komentáře `ZMĚNA`:

In [ ]:
class NgramTagger:

    # ZMĚNA: nový argument backoff, který obsahuje tagger, který se má
    # použít ve chvíli, kdy tagování pomocí stávajícího taggeru selže
    def __init__(self, data, backoff):
        cfd = nltk.ConditionalFreqDist()
        for sent in data:
            for i, (word, tag) in enumerate(sent):
                if tag is None:
                    continue
                ctx = self.get_context(i, word, sent)
                cfd[ctx][tag] += 1
        self._model = cfd
        # ZMĚNA: backoff tagger je též potřeba uložit na self
        self._backoff = backoff

    # ZMĚNA: nová metoda .tag(); není abstraktní, je vymyšlená tak, aby
    # fungovala v kombinaci s jakýmkoli n-gramovým taggerem
    def tag(self, sent):
        # začneme tím, že si větu necháme označkovat backoff taggerem a tagy
        # si schováme, abychom si z nich kdykoli v případě potřeby mohli
        # vytáhnout tag pro pozici, kterou se nám nepodařilo označkovat
        # stávajícím taggerem
        backoff = [tag for _, tag in self._backoff.tag(sent)]
        tagged_sent = []
        for i, word in enumerate(sent):
            # pro každé slovo ve větě se pokusíme určit tag...
            try:
                # určíme kontext metodou odpovídající příslušnému konkrétnímu
                # n-gramovému taggeru; POZOR: určení kontextu musí probíhat
                # na základě *již označkované* části věty, tj. tagged_sent,
                # ne sent
                ctx = self.get_context(i, word, tagged_sent)
                # a na základě kontextu vytáhneme z modelu odpovídající tag
                tag = self._model[ctx].max()
            # ... ale když se to nepodaří, nic se neděje: použijeme backoff
            except ValueError:
                tag = backoff[i]
            tagged_sent.append((word, tag))
        return tagged_sent

    def get_context(self, i, word, sent):
        raise NotImplementedError

class UnigramTagger(NgramTagger):

    def get_context(self, i, word, sent):
        return word.lower()

class BigramTagger(NgramTagger):

    def get_context(self, i, word, sent):
        prev_tag = None if i == 0 else sent[i-1][1]
        return prev_tag, word.lower()

Všimněte si že **jediná třída, ve které bylo potřeba učinit změny, je abstraktní třída `NgramTagger`**. Derivované třídy `UnigramTagger` a `BigramTagger` mohly zůstat stejné jako dřív a najednou "zadarmo" získaly novou funkcionalitu (neslouží už jenom k trénování modelů, umí i otagovat větu). V tom je výhoda (dobře rozmyšleného) objektově orientovaného programování.

Jistá nevýhoda tkví naopak v tom, že je všechno potřeba trochu víc rozmýšlet předem, zejména vztahy mezi jednotlivými třídami, upravit algoritmy tak, aby většina metod mohla být sdílených, vytipovat klíčové metody, které bude muset každá podřazená třída mít vlastní, atp.

Za pozornost stojí ještě jedna drobnost: nikde není řečeno, co přesně musí být v atributu `self._backoff` uloženo za objekt. Podle toho, jak ho v metodě `.tag()` používáme, můžeme odvodit, že stačí, aby měl následující vlastnosti: musí **sám disponovat vlastní metodou `.tag()`**, která jako argument **přijme seznam řetězců** a **vrátí stejně dlouhý seznam dvojic**. To znamená, že jako defaultní tagger můžeme klidně použít třídu `nltk.DefaultTagger`, která tyto podmínky splňuje, nemusíme definovat svou vlastní:

In [ ]:
dt = nltk.DefaultTagger("N")
ut = UnigramTagger(train, backoff=dt)
bt = BigramTagger(train, backoff=ut)

In [ ]:
bt.tag(sent)

Jinými slovy, naše n-gramové taggery jsou (alespoň v tomto ohledu) interoperabilní s taggery definovanými v NLTK. Této filozofii práce s různými typy objektů, kdy nás tolik nezajímá, jaký přesně typ objektu máme k dispozici (`NgramTagger`? `FrequencyDistribution`? `Cat`? `Dog`?), ale co ten konkrétní objekt umí (disponuje metodou `.tag()`?), se říká [*duck typing*](https://en.wikipedia.org/wiki/Duck_typing):

> If it walks like a duck and it quacks like a duck, then it must be a duck. (Turns out it's a goose? Close enough, because geese quack too.)